
## DATA ANALYSIS JURA : Eco-acoustics week
MOBI

Author : Paul Peyret (13-11-2022)

Group : Jure Zeleznik,
Loréna Boisseau,
Annaïs Soares,
Paul Peyret

This script analyzes the data recorded using 4 Audiomoths on a transect of different altitude in the forest of Risoux (Jura).

This notebook as 3 parts: 
- PART 1: ACOUSTIC INDICES CALCULATION
This part will explore different folders and calculate acousic indices from scikit-maad toolbox and store it into a csv file
- PART 2: INDICES ANALYSIS
This part load the acoustic indices calculated in PART 1 and do some statistical analysis of a given index on the different points of the transect
- PART 3: LONG TERM SPECTROGRAM CALCULATION
This part loads all the audio files again in a given point of the transect and calculates the long term average spectrogram (LTSA) at this recording position.

Important note: The data were recorded on devices using UTC time. To get local time you must add 2 hours to the datetime string indicated in the name of the wave file.

####Prerequisit :

Install Python 3.10
1) Install packages: 
`$ pip install -r requirements.txt`
2) Copy recording folders in a "data/" subfolder in your working directory
3) Outputs : 
    df_indices.csv csv file containing the dataframe with the indicators
    out_plot shall be a subfolder of your working directory



### PART 1 : CALCULATE ACOUSTIC INDICES

In [2]:
# import libraries
import os
import glob
import pandas as pd
import scipy as sp
import numpy as np
from datetime import datetime, timedelta
from maad import sound, features, spl, util
from maad.util import plot_features_map
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#CALCULATE INDICES (time consuming)
# Prepare dataframe for reading
# # ----------  set parameters ------------
# User Input
usefull_dates_P1=["2022-10-05","2022-10-06"]  # we select only one day
allrecPos=['4.1','1.2','10.2','12.2']

# ----------  end of set parameters ------------


# List files and build dataframe
df=pd.DataFrame()
path = os.getcwd()
fileformat='%Y%m%d_%H%M%S.WAV'

# loop over folders to agregate filenames in a big dataframe
for config in allrecPos:
    dftmp=pd.DataFrame()
    recPos=config
    subfolder='mobi_am_'+recPos
    print("Processing "+subfolder+"...")

    # Add all filename in the directory in a dataframe
    dftmp=pd.DataFrame({"file":glob.glob(os.path.join(path, 'data/'+subfolder+'/*.WAV'))})
    # another way to do this is filelist = glob(datapath+'/**/*.WAV', recursive = True)'

    dftmp['recpos']=recPos
    df=pd.concat([df,dftmp])

# Extract filename from fullpath
df['filename']=df["file"].apply(lambda x: os.path.basename(x))
# Extract Date and time from filename
df['DateUTC']=df["filename"].apply(lambda x: datetime.strptime(x, fileformat))
df['Date']=df["DateUTC"].apply(lambda x: x+timedelta(hours=2))

# Set date as index
df.set_index('Date', inplace=True)
df=df.sort_values(['recpos','Date'])

# Remove not usable dates from P1  & Remove not usable dates from P2
mask=((df.index > usefull_dates_P1[0]) & (df.index < usefull_dates_P1[1]))
df=df[mask]

print(df.head())
# 

In [ ]:

#%% BATCH COMPUTING OF ACOUSTIC INDICES
# RUN ONLY ONCE TO GENERATE THE "df_indices.csv" FILE

# initialize dataframes and indices
df_indices = pd.DataFrame() 
df_indices_per_bin = pd.DataFrame()
N=df.shape[0]
id=0

for index, row in df.iterrows() :
    id=id+1
    # get the full filename of the corresponding row
    fullfilename = row['file']
    # Save file basename
    path, filename = os.path.split(fullfilename)
    print ('\n**************************************************************')
    print(str(id)+"/"+str(N))
    print (filename)

    #### Load the original sound (16bits) and get the sampling frequency fs
    try :
        wave,fs = sound.load(filename=fullfilename, channel='left', detrend=True, verbose=False)

    except:
        # Delete the row if the file does not exist or raise a value error (i.e. no EOF)
        df.drop(index, inplace=True)
        continue

    """ =======================================================================
                     Computation in the time domain
    ========================================================================"""

    # Parameters of the audio recorder. This is not a mandatory but it allows
    # to compute the sound pressure level of the audio file (dB SPL) as a
    # sonometer would do.
    S = -18         # -18dBV (Audiomoth)
    G = 15       # Amplification gain Medium Gain

    # compute all the audio indices and store them into a DataFrame
    # dB_threshold and rejectDuration are used to select audio events.
    df_audio_ind = features.all_temporal_alpha_indices(wave, fs,
                                          gain = G, sensibility = S,
                                          dB_threshold = 3, rejectDuration = 0.01,
                                          verbose = False, display = False)

    """ =======================================================================
                     Computation in the frequency domain
    ========================================================================"""

    # Compute the Power Spectrogram Density (PSD) : Sxx_power
    Sxx_power,tn,fn,ext = sound.spectrogram (wave, fs, window='hann',
                                             nperseg = 1024, noverlap=1024//2,
                                             verbose = False, display = False,
                                             savefig = None)

    # compute all the spectral indices and store them into a DataFrame
    # flim_low, flim_mid, flim_hi corresponds to the frequency limits in Hz
    # that are required to compute somes indices (i.e. NDSI)
    # if R_compatible is set to 'soundecology', then the output are similar to
    # soundecology R package.
    # mask_param1 and mask_param2 are two parameters to find the regions of
    # interest (ROIs). These parameters need to be adapted to the dataset in
    # order to select ROIs
    df_spec_ind, df_spec_ind_per_bin = features.all_spectral_alpha_indices(Sxx_power,
                                                            tn,fn,
                                                            flim_low = [0,1500],
                                                            flim_mid = [1500,8000],
                                                            flim_hi  = [8000,48000],
                                                            gain = G, sensitivity = S,
                                                            verbose = False,
                                                            R_compatible = 'soundecology',
                                                            mask_param1 = 6,
                                                            mask_param2=0.5,
                                                            display = False)

    """ =======================================================================
                     Create a dataframe
    ========================================================================"""
    # First, we create a dataframe from row that contains the date and the
    # full filename. This is done by creating a DataFrame from row (ie. TimeSeries)
    # then transposing the DataFrame.
    df_row = pd.DataFrame(row)
    df_row =df_row.T
    df_row.index.name = 'Date'
    df_row = df_row.reset_index()

    # add scalar indices into the df_indices dataframe
    df_indices = df_indices.append(pd.concat([df_row,
                                              df_audio_ind,
                                              df_spec_ind], axis=1))
    # add vector indices into the df_indices_per_bin dataframe
    #df_indices_per_bin = df_indices_per_bin.append(pd.concat([df_row,
    #                                                          df_spec_ind_per_bin], axis=1))
# Set back Date as index
df_indices = df_indices.set_index('Date')
#df_indices_per_bin = df_indices_per_bin.set_index('Date')

# SAVE FILE to CSV
df_indices.to_csv('df_indices.csv')
print('File saved')

# %% PLOT FEATURE MAP
SPECTRAL_FEATURES=['MEANf','VARf','SKEWf','KURTf','NBPEAKS','LEQf',
'ENRf','BGNf','SNRf','Hf', 'EAS','ECU','ECV','EPS_KURT','EPS_SKEW','ACI',
'NDSI','rBA','AnthroEnergy','BioEnergy','BI','ROU','ADI','AEI','LFC','MFC','HFC',
'ACTspFract','ACTspCount','ACTspMean', 'EVNspFract','EVNspMean','EVNspCount',
'TFSD','H_Havrda','H_Renyi','H_pairedShannon', 'H_gamma', 'H_GiniSimpson','RAOQ',
'AGI','ROItotal','ROIcover']

TEMPORAL_FEATURES=['ZCR','MEANt', 'VARt', 'SKEWt', 'KURTt',
               'LEQt','BGNt', 'SNRt','MED', 'Ht','ACTtFraction', 'ACTtCount',
               'ACTtMean','EVNtFraction', 'EVNtMean', 'EVNtCount']

# plot feature map
plot_features_map(df_indices[SPECTRAL_FEATURES], mode='24h')
plot_features_map(df_indices[TEMPORAL_FEATURES], mode='24h')

### PART 2: INDICES ANALYSIS : ACI

In [ ]:
# PREPARE DATAFRAME
import pandas as pd

#%% =================== Load file===========================
df_indices=pd.read_csv("df_indices.csv")
df_indices.drop('file', inplace=True, axis=1) # drop path column

# Retrieve datetime index
df_indices["Date"]=pd.to_datetime(df_indices['Date'])
df_indices['Time'] = df_indices['Date'].dt.strftime('%H:%M')

df_indices.set_index('Date', inplace=True)
df_indices=df_indices.sort_values(['recpos','Date'])

# Calculate missing H indice
df_indices["H"]=df_indices["Ht"]*df_indices["Hf"]
# Drop EPS which is not calculated
df_indices.drop('EPS',inplace=True, axis=1)


In [ ]:
# VIOLIN PLOT : ACI
import seaborn as sns

# %% ==================Get the Subset of the data frame ==========
# ================= user input ===============
usefull_dates=["2022-10-05","2022-10-06"] #
sel_recopos=['4.1','1.2','10.2','12.2']#['12.2']#['4.1','1.2','10.2','12.2']##['4.1','1.2','10.2','12.2']#["CH9"] # 
time_sample_per_hour=12 # Integer (there is maximum 12 samples per hour)
indexName='ACI' # Name of the index to plot
outdir='out_plot/'#'./out_plot/' # your output directory name
# ===============end user input ==============

#%%=========Violin Plot===========
fig=plt.figure()
df=df_indices
df['recposP'] = df['recpos'].astype('category')
df['recposP'] = df['recposP'].map({'4.1': 'P4', '1.2': 'P3', '10.2': 'P2', '12.2': 'P1'})
dftmp=df[['recposP',indexName]]
dftmp['recposP']=dftmp['recposP'].cat.as_ordered()
dftmp['recposP']=dftmp["recposP"].cat.reorder_categories(["P1", "P2", "P3","P4"],ordered=True)
dftmp=dftmp.sort_values(by='recposP')

my_pal = {"P1": "r", "P2": "orange", "P3": "yellow","P4": "green"}

sns.violinplot(x=dftmp["recposP"], y=dftmp[indexName],palette=my_pal)
if not outdir == "":
    fig=plt.gcf()
    fig.savefig(outdir+'violin_'+indexName+'.png')


In [ ]:
# STATISTICS
import scipy.stats as stats
from itertools import combinations

combi=list(combinations(sel_recopos,2))
df_test=df[['recpos',indexName]]
print('\nMann Whitney U test (Wilcoxon rank sum test)')
print(indexName)
for c in combi:
    print('\nComparing '+c[0]+' and '+c[1])
    dfx1=df_test[df_test['recpos']==c[0]]
    dfy1=df_test[df_test['recpos']==c[1]]
    # perform two-sided test. You can use 'greater' or 'less' for one-sided test
    Statvalue,p=stats.mannwhitneyu(x=dfx1[indexName], y=dfy1[indexName], alternative = 'two-sided')
    print(f"\nStat = {Statvalue:.1f} \t p = {p}")
    # output

### PART 3: Long Term Spectrogram

In [ ]:
# PREPARE DATA
from maad import sound, features, spl, util
from maad.util import plot_features_map
import pandas as pd
import matplotlib.pyplot as plt

# ----------  set parameters ------------
# For this experiment, we used a AudioMoth
S = -18         # Sensibility of the microphone : -18dBV (Audiomoth)   
G = 15       # Total amplification gain in dB (audiomoth medium Gain)
VADC = 2        # Voltage range of the analog to digital converter (ADC)

fstep=500 # Frequency step in the LTSA
fmax=60000 # Maximum frequency of the LTSA
Nfft=512 # Nfft for the spectrogram calculation

# Selected recording position (change the value to calculate for another recording position)
sel_recopos=['10.2']#['4.1','1.2','10.2','12.2']#["CH9"] # 
# ----------  end of set parameters ------------


#%%
# First, read df_indices to get the file names
df=pd.read_csv("df_indices.csv")
# Keep only columns of interest
df=df[['Date','file','filename',"recpos"]]

# Set date as index
df["Date"]=pd.to_datetime(df['Date'])
df['Time'] = df['Date'].dt.strftime('%H:%M')
df.set_index('Date', inplace=True)
df=df.sort_values(['recpos','Date'])

df['recpos'] = df['recpos'].map(str)  # convert number to strings
df=df[df['recpos'].isin(sel_recopos)] 
df=df.iloc[:,:]



In [ ]:
# CALCULATE LTSA
# Load and preprocess audio
# -------------------------
# Then we process all the files found in the directory /indices.
# Initialisation of an empty dataframe df_spl to store all the dB SPL values 
# extracted from the whole audio dataset.

df_spl=pd.DataFrame()

# Main loop to go through all audio files
for index, row in df.iterrows() : 
    
    # initialisation of an empty list to store the dB SPL of the current
    # audio recording.
    leq_list = []
    
    # get the full filename of the corresponding row
    fullfilename = row['file']
    # Save file basename
    path, filename = os.path.split(fullfilename)
    print ('\n**************************************************************')
    print (filename)
    
    #### Load the original sound (16bits, only left channel) and get the sampling 
    # frequency fs
    try :
        wave,fs = sound.load(filename=fullfilename, channel='left', detrend=True, verbose=False)

    except:
        print('\n !! file not found !!')
        # Delete the row if the file does not exist or raise a value error (i.e. no EOF)
        df.drop(index, inplace=True)
        continue
    
    """ =======================================================================
                     Computation in the frequency domain 
    ========================================================================"""
 
    # Compute the Power Spectrogram Density (PSD) : Sxx_power
    Sxx_power,tn,fn,ext = sound.spectrogram (wave, fs, window='hann', 
                                             nperseg = Nfft, noverlap=Nfft/2, 
                                             verbose = False, display = False, 
                                             savefig = None)   
    
    
    #### Average PSD (It's a mandatory to compute the mean on the PSD for 
    # energy conservation)
    mean_PSD = np.mean(Sxx_power, axis = 1)
    flim=list(range(0,fmax+1,fstep))
    labels_leq=[]
    for id in range(0,len(flim)-1,1):
        #### Compute the Leq of each frequency band
        leq_list+=[spl.psd2leq(mean_PSD[util.index_bw(fn,(flim[id],flim[id+1]))], 
                                gain=G, 
                                sensitivity=S, 
                                Vadc=VADC)]
        labels_leq+=[str(flim[id]/1000)+'-'+str(flim[id+1]/1000)+'kHz']
    
    #### Create a dataframe from the list
    df_leq = pd.DataFrame([leq_list],
                          columns = labels_leq)
    
    #%
    """ =======================================================================
                     Create a dataframe 
    ========================================================================"""
    #### We create a dataframe from row that contains the date and the 
    # full filename. This is done by creating a DataFrame from row (ie. TimeSeries)
    # then transposing the DataFrame. 
    df_row = pd.DataFrame(row)
    df_row =df_row.T
    df_row.index.name = 'Date'
    df_row = df_row.reset_index()

    #### add Leq values into the df_spl dataframe
    df_spl = df_spl.append(pd.concat([df_row, df_leq], axis=1))

#### When the loop ends, set Date as index
df_spl = df_spl.set_index('Date')



In [ ]:
# PLOT LTSA
outdir='out_plot/'

import seaborn as sns
# plot using a color palette
fig=plt.figure(figsize=[8,5])
df_spl_rev = df_spl.iloc[:,::-1].iloc[:,:-4] # reverse dataframe and select only columns with SPL data

ax=sns.heatmap(df_spl_rev.iloc[:,:].T,linewidths=0.0,rasterized=True,vmin=20, vmax=70) #cmap="YlGnBu")
xlabelid=list(range(0,len(df_spl_rev.index),1))
ylabelid=list(range(0,len(flim)+1,1))
plt.yticks(ylabelid[0::5],map(str, list(reversed(flim[0::5]))))  # Set text labels.
plt.xticks(xlabelid[11::12],df_spl_rev.index[11::12].strftime('%H:%M'))  # Set text labels.
plt.title('LTSA '+sel_recopos[0])

if not outdir == "":
    fig=plt.gcf()
    fig.savefig(outdir+'_'+'LTSA'+'_'.join(sel_recopos)+'.png')